# DDP_parsing prediction and f1 score with published model 

Model downloaded from https://drive.google.com/file/d/1ytLubJSWP1eeZDpQekWkWmJ1D0ODCvou/view


In [1]:
import pickle
import json
import pandas as pd

map_relations = {'Comment': 0, 'Contrast': 1, 'Correction': 2, 'Question-answer_pair': 3, 'QAP': 3, 'Parallel': 4, 'Acknowledgement': 5,
            'Elaboration': 6, 'Clarification_question': 7, 'Conditional': 8, 'Continuation': 9, 'Result': 10, 'Explanation': 11,
            'Q-Elab': 12, 'Alternation': 13, 'Narration': 14, 'Background': 15, 'Break': 16}

In [37]:
pred_file =  open("outputs/stac_predictions_attach.pkl",'rb')
stac_predictions = pickle.load(pred_file)
pred_rel_file =  open("outputs/stac_predictions_relation.pkl",'rb')
stac_rel_predictions = pickle.load(pred_rel_file)
f = open("data/stac_data/test_data.json")
stac_test = json.load(f)

print(stac_rel_predictions[0])
print(stac_predictions[0])

# delete the root node from the predictions and regroup
test_stac_pred = [[(p[1]-1,p[0]-1) for p in dialog_pred[1:]] for dialog_pred in stac_predictions if len(dialog_pred) >1]
test_stac_pred_rel = [[(stac_predictions[j][i][1]-1,stac_predictions[j][i][0]-1,stac_rel_predictions[j][i]) for i in range(1,len(stac_predictions[j]))] for j in range(len(stac_predictions)) if len(stac_predictions[j]) >1]

test_stac_truth = []
for dialogue in stac_test:
    truth = []
    if len(dialogue['edus'])==1:
        continue
    for edu in dialogue['relations']:
        tup = (edu['x'],edu['y'],map_relations[edu['type']])
        truth += [tup]
    test_stac_truth += [truth]

[16, 3, 3, 5]
[[1, 0], [2, 1], [3, 1], [4, 3]]


In [44]:
# f1 for attachments - code for f1 computation from shi and huang 
# cnt_pred
cnt_pred = []
for dialog in test_stac_pred:
    cnt_pred += [len(dialog)+1]
cnt_pred = sum(cnt_pred)

# cnt_golden
cnt_golden = []
for i,dialog in enumerate(stac_test):  
    cnt_g = len(dialog['relations'])
    cnt = [0] * len(dialog["edus"])
    for r in dialog["relations"]:
        cnt[r["y"]] += 1
    for j in range(len(dialog["edus"])):
        if cnt[j] == 0:
            cnt_g += 1
    cnt_golden += [cnt_g]
cnt_golden = sum(cnt_golden)

cnt_correct = []
for i, dialog_pred in enumerate(test_stac_pred):
    val = 0
    truth = [(p[0], p[1]) for p in test_stac_truth[i]]
    for pred in dialog_pred : 
        if pred in truth:
            val += 1
    cnt_correct += [val+1]
cnt_correct = sum(list(cnt_correct))
print('nb of correct predictions : ',cnt_correct)
print('nb of attachments in the gold data : ', cnt_golden)
print('nb of predictions : ', cnt_pred)
precision = cnt_correct*1.0/cnt_pred*1.0
recall = cnt_correct*1.0/cnt_golden*1.0

f1 = 2*( precision * recall / (precision + recall))
print('F1 score :', f1)


# cnt_golden-cnt_pred = 89
precision = cnt_correct*1.0/cnt_pred*1.0
recall = cnt_correct*1.0/(cnt_golden-89)*1.0
f1_bis = 2*( precision * recall / (precision + recall))
print('F1 score 1 parent per edu :', f1_bis)

nb of correct predictions :  810
nb of attachments in the gold data :  1243
nb of predictions :  1154
F1 score : 0.6758448060075093
F1 score 1 parent per edu : 0.7019064124783362


In [45]:
# f1 for relations - code for f1 computation from shi and huang 
# cnt_pred
cnt_pred = []
for dialog in test_stac_pred:
    cnt_pred += [len(dialog)+1]
cnt_pred = sum(cnt_pred)

# cnt_golden
cnt_golden = []
for i,dialog in enumerate(stac_test):  
    cnt_g = len(dialog['relations'])
    cnt = [0] * len(dialog["edus"])
    for r in dialog["relations"]:
        cnt[r["y"]] += 1
    for j in range(len(dialog["edus"])):
        if cnt[j] == 0:
            cnt_g += 1
    cnt_golden += [cnt_g]
cnt_golden = sum(cnt_golden)

cnt_correct = []
for i, dialog_pred in enumerate(test_stac_pred_rel):
    val = 0
    truth = [(p[0], p[1], p[2]) for p in test_stac_truth[i]]
    for pred in dialog_pred : 
        if pred in truth:
            val += 1
    cnt_correct += [val+1]
cnt_correct = sum(list(cnt_correct))
print('nb of correct predictions : ',cnt_correct)
print('nb of attachments in the gold data : ', cnt_golden)
print('nb of predictions : ', cnt_pred)
precision = cnt_correct*1.0/cnt_pred*1.0
recall = cnt_correct*1.0/cnt_golden*1.0

f1 = 2*( precision * recall / (precision + recall))
print('F1 score :', f1)

precision = cnt_correct*1.0/cnt_pred*1.0
recall = cnt_correct*1.0/(cnt_golden-89)*1.0
f1_bis = 2*( precision * recall / (precision + recall))
print('F1 score 1 parent per edu :', f1_bis)

nb of correct predictions :  584
nb of attachments in the gold data :  1243
nb of predictions :  1154
F1 score : 0.4872757613683771
F1 score 1 parent per edu : 0.5060658578856152
